In [ ]:
def plot_fmri_signal(fmri_file, voxel=None):

    # Load the fMRI data
    fmri_data = nib.load(fmri_file).get_fdata()  # Shape: (H, W, D, T)
    time_points = fmri_data.shape[-1]

    if voxel is not None:
        # Extract the signal for a specific voxel
        signal = fmri_data[voxel[0], voxel[1], voxel[2], :]
        title = f"Voxel Signal at {voxel}"
    else:
        # Average across all voxels
        fmri_flat = fmri_data.reshape(-1, time_points)  # Shape: (H*W*D, T)
        signal = np.median(fmri_flat, axis=0)  # Mean across all voxels
        title = "Average Signal Across All Voxels"

    # Plot the signal
    plt.figure(figsize=(10, 6))
    plt.plot(signal, label='fMRI Signal')
    plt.title(title)
    plt.xlabel('Time Points')
    plt.ylabel('Signal Intensity')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01035_2_T1.nii.gz"

# Plot signal for a specific voxel
plot_fmri_signal(fmri_file, voxel=(30, 30, 20))  # Replace with a valid voxel coordinate

# Plot average signal across all voxels
plot_fmri_signal(fmri_file)


In [ ]:
def view_fmri_slice(fmri_data, time_point, axis=2, slice_index=None):

    # Load the fMRI data
    fmri_shape = fmri_data.shape

    # Validate inputs
    if time_point < 0 or time_point >= fmri_shape[-1]:
        raise ValueError(f"Time point must be in the range [0, {fmri_shape[-1] - 1}]")
    if axis < 0 or axis > 2:
        raise ValueError("Axis must be 0 (sagittal), 1 (coronal), or 2 (axial)")

    # Select the time point data
    volume = fmri_data[..., time_point]  # Shape: (H, W, D)

    # Determine the slice index
    if slice_index is None:
        slice_index = volume.shape[axis] // 2  # Default to the middle slice
    if slice_index < 0 or slice_index >= volume.shape[axis]:
        raise ValueError(f"Slice index must be in the range [0, {volume.shape[axis] - 1}]")

    # Extract the specific slice
    if axis == 0:  # Sagittal slice
        slice_data = volume[slice_index, :, :]
    elif axis == 1:  # Coronal slice
        slice_data = volume[:, slice_index, :]
    elif axis == 2:  # Axial slice
        slice_data = volume[:, :, slice_index]

    # Plot the slice
    plt.figure(figsize=(8, 8))
    plt.imshow(slice_data.T, cmap='gray', origin='lower')
    mn = np.mean(slice_data)
    plt.title(f"Time Point {time_point}, Slice {slice_index} (Axis {axis}) -- AVG {mn}")
    plt.colorbar(label="Signal Intensity")
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.show()

# Example usage
fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01035_2_T1.nii.gz"

fmri_data = nib.load(fmri_file).get_fdata()  # Shape: (H, W, D, T)

print(fmri_data.shape)


In [ ]:
# View an axial slice at time point 10
view_fmri_slice(fmri_data, time_point=0, axis=2, slice_index=15)


In [2]:
import nibabel as nib
import numpy as np
from scipy.io import savemat

def extract_timeseries_to_mat(input_path, output_path, remove_time_points=0):
    """
    Extracts voxel-wise time series from an fMRI scan and saves it to a .mat file.

    Parameters:
    - input_path (str): Path to the fMRI file (.nii or .nii.gz).
    - output_path (str): Path to save the .mat file.
    - remove_time_points (int): Number of initial time points to remove.

    Returns:
    - None
    """
    # Load fMRI data
    img = nib.load(input_path)
    data = img.get_fdata()  # Shape: (X, Y, Z, T)

    # Remove initial time points if specified
    if remove_time_points > 0:
        data = data[..., remove_time_points:]

    # Reshape the data to (num_voxels, time_points)
    num_voxels = np.prod(data.shape[:-1])  # Total number of voxels
    time_points = data.shape[-1]  # Number of remaining time points
    reshaped_data = data.reshape((num_voxels, time_points))

    # Save to .mat file
    mat_dict = {"timeseries": reshaped_data}
    savemat(output_path, mat_dict)

    print(f"Time series extracted and saved to {output_path}")

# Example usage
fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01035_2_T1.nii.gz"

output_mat_file = "timeseries.mat"  # Path to save the .mat file

extract_timeseries_to_mat(fmri_file, output_mat_file, remove_time_points=0)


Time series extracted and saved to timeseries.mat


In [5]:
import nibabel as nib
import numpy as np
from scipy.io import savemat

def save_fmri_as_4d_mat(input_path, output_path, remove_time_points=0):
    """
    Saves fMRI data as a 4D .mat file.

    Parameters:
    - input_path (str): Path to the fMRI file (.nii or .nii.gz).
    - output_path (str): Path to save the .mat file.
    - remove_time_points (int): Number of initial time points to remove.

    Returns:
    - None
    """
    # Load fMRI data
    img = nib.load(input_path)
    data = img.get_fdata()  # Shape: (X, Y, Z, T)

    # Remove initial time points if specified
    if remove_time_points > 0:
        data = data[..., remove_time_points:]  # Slice the time dimension

    # Save the data to a .mat file as a 4D array
    mat_dict = {"fmri_4d": data}
    savemat(output_path, mat_dict)

    print(f"4D fMRI data saved to {output_path} with shape {data.shape}")

# Example usage
# fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01035_2_T1.nii.gz"
fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01060_2_T1.nii.gz"
output_mat_file = "fmri_data_4d.mat"  # Path to save the .mat file

save_fmri_as_4d_mat(fmri_file, output_mat_file, remove_time_points=5)


4D fMRI data saved to fmri_data_4d.mat with shape (64, 64, 26, 430)


In [6]:
import nibabel as nib
import numpy as np
from scipy.io import savemat

def save_cvr_as_3d_mat(input_path, output_path, remove_time_points=0):
    
    # Load fMRI data
    img = nib.load(input_path)
    data = img.get_fdata()  # Shape: (X, Y, Z, T)

    
    # Save the data to a .mat file as a 4D array
    mat_dict = {"cvr_3d": data}
    savemat(output_path, mat_dict)

    print(f"3D CVR data saved to {output_path} with shape {data.shape}")

# Example usage
# fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01035_2_T1.nii.gz"
cvr_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/CVR_MAPS/registered/training/SF_01035_CVR_2_T1.nii.gz"
output_mat_file = "cvr_data_3d.mat"  # Path to save the .mat file

save_fmri_as_4d_mat(cvr_file, output_mat_file)


4D fMRI data saved to cvr_data_3d.mat with shape (64, 64, 26)


In [ ]:

def load_fmri_data(file_path):
    """
    Load and preprocess fMRI data for prediction.

    Args:
        file_path (str): Path to the 4D fMRI file (.nii or .nii.gz).

    Returns:
        tuple: (time_series_data, spatial_shape)
               - time_series_data (np.ndarray): Shape (H*W*D, T).
               - spatial_shape (tuple): Original spatial dimensions (H, W, D).
    """
    fmri_data = nib.load(file_path).get_fdata()  # Shape: (H, W, D, T)
    spatial_shape = fmri_data.shape[:-1]  # (H, W, D)
    time_points = fmri_data.shape[-1]  # T

    # Reshape to voxel-wise time-series
    time_series_data = fmri_data.reshape(-1, time_points)  # Shape: (H*W*D, T)
    return time_series_data, spatial_shape

def predict_cvr_map(model, time_series_data, input_size, device):
    """
    Predict CVR values for each voxel based on fMRI time-series data.

    Args:
        model (torch.nn.Module): Trained PyTorch model.
        time_series_data (np.ndarray): Shape (H*W*D, T), voxel-wise time-series.
        input_size (int): Size of the input to the model (T).
        device (torch.device): Device to run predictions on.

    Returns:
        np.ndarray: Predicted CVR map, shape (H*W*D).
    """
    model.eval()
    model.to(device)

    # Prepare the data for prediction
    time_series_tensor = torch.tensor(time_series_data, dtype=torch.float32).unsqueeze(1).to(device)  # Shape: (N, 1, T)

    # Perform predictions
    with torch.no_grad():
        predictions = model(time_series_tensor).cpu().numpy()  # Shape: (N, 1)
    return predictions.squeeze()  # Shape: (H*W*D)

def visualize_cvr_map(cvr_map, spatial_shape, slice_index, axis=2):
    """
    Visualize a slice of the predicted CVR map.

    Args:
        cvr_map (np.ndarray): Predicted CVR map, flattened (H*W*D).
        spatial_shape (tuple): Original spatial shape (H, W, D).
        slice_index (int): Index of the slice to visualize.
        axis (int): Axis for slicing (0=x, 1=y, 2=z). Default is 2 (axial view).
    """
    cvr_map_reshaped = cvr_map.reshape(spatial_shape)  # Reshape to (H, W, D)

    # Extract the desired slice
    if axis == 0:  # Sagittal slice
        slice_data = cvr_map_reshaped[slice_index, :, :]
    elif axis == 1:  # Coronal slice
        slice_data = cvr_map_reshaped[:, slice_index, :]
    elif axis == 2:  # Axial slice
        slice_data = cvr_map_reshaped[:, :, slice_index]

    # Plot the slice
    plt.figure(figsize=(8, 8))
    plt.imshow(slice_data.T, cmap='hot', origin='lower')
    plt.title(f"Predicted CVR Map (Slice {slice_index}, Axis {axis})")
    plt.colorbar(label="CVR Value")
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.show()

# Example Usage
fmri_file = "/Users/muhammadmahajna/workspace/research/data/cvr_est_project/func/registered/main_data/training/SF_01035_2_T1.nii.gz"

model_path = "best_model.pth"  # Path to the trained model
input_size = 435  # Number of time points

# Load and preprocess fMRI data
time_series_data, spatial_shape = load_fmri_data(fmri_file)

# Load the trained model
model = CNN1DModel(input_size=input_size)
model.load_state_dict(torch.load(model_path))

# Predict the CVR map
predicted_cvr_map = predict_cvr_map(model, time_series_data, input_size, device)

# Visualize a slice of the predicted CVR map
visualize_cvr_map(predicted_cvr_map, spatial_shape, slice_index=10, axis=2)
